In [1]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess
from datetime import date
from joblib import Parallel, delayed

2023-04-13 11:46:28.019829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 11:46:28.125855: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-13 11:46:28.443225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-04-13 11:46:28.443269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.1...


In [2]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes'

In [3]:
if not os.path.exists('All_vids'):
        os.makedirs('All_vids')
os.chdir('All_vids')

In [4]:
#enter path of folder (including folder name) containing all your videos of interest
vid_dir = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Videos'
to_dir = os.path.join(os.getcwd(), os.path.basename(vid_dir))
shutil.move(vid_dir, to_dir)

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos'

In [5]:
os.path.dirname(to_dir)

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [6]:
#Soring converted videos in a new folder
if not os.path.exists('Conv_vids'):
        os.makedirs('Conv_vids')

In [7]:
input_file = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera.mov'
output_file = os.path.splitext(os.path.basename(input_file))[0].lstrip('.') 
output_file
os.path.join(os.path.dirname(os.path.dirname(input_file)), '/Conv_vids', output_file +'_conv.mp4')

'/Conv_vids/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera_conv.mp4'

In [8]:
#Converting .mov videos to .mp4
def conv_mp4(input_file):
    out_name = os.path.splitext(input_file)[0].lstrip('.') 
    print(out_name)
    output_file = out_name + '.mp4'
    # print(input_file)
    # print(output_file)
    # Run FFmpeg command to convert MOV to MP4
    command = f"ffmpeg -i {input_file} -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 128k {output_file}"
    subprocess.call(command, shell=True)
    new = os.path.join(os.path.dirname(os.path.dirname(output_file)), 'Conv_vids', os.path.basename(output_file))
    shutil.move(output_file, new)
    # print('Convertion complete - ', os.path.basename(output_file))

In [9]:
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Videos')):
    if file.endswith(".mov"):
        # print(file)
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Videos', file))
        conv_mp4(path)
        # print(path)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/vin_20210604_0907_XBI19_reversal_learning_foraging_SC3_DCS_front_camera


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/nil_20210614_XBI19


frame=232398 fps=1490 q=-1.0 Lsize=  185185kB time=01:04:33.25 bitrate= 391.7kbits/s speed=24.8x    
video:182469kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.488505%
[libx264 @ 0x55b27b9a5180] frame I:962   Avg QP:21.70  size: 56129
[libx264 @ 0x55b27b9a5180] frame P:95337 Avg QP:23.99  size:  1264
[libx264 @ 0x55b27b9a5180] frame B:136099 Avg QP:31.20  size:    91
[libx264 @ 0x55b27b9a5180] consecutive B-frames:  1.4% 61.0%  1.5% 36.1%
[libx264 @ 0x55b27b9a5180] mb I  I16..4: 14.1% 32.1% 53.8%
[libx264 @ 0x55b27b9a5180] mb P  I16..4:  0.8%  0.9%  0.2%  P16..4: 18.8%  1.0%  1.4%  0.0%  0.0%    skip:76.9%
[libx264 @ 0x55b27b9a5180] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  5.7%  0.0%  0.0%  direct: 0.0%  skip:94.2%  L0:17.2% L1:82.6% BI: 0.2%
[libx264 @ 0x55b27b9a5180] 8x8 transform intra:43.0% inter:72.3%
[libx264 @ 0x55b27b9a5180] coded y,uvDC,uvAC intra: 53.1% 76.8% 39.0% inter: 1.9% 5.9% 0.1%
[libx264 @ 0x55b27b9a5180] i16 v,h,dc,p: 20% 44% 14% 2

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/ken_20210506_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera


...

More than 1000 frames duplicated   17664kB time=00:01:14.27 bitrate=1948.2kbits/s dup=939 drop=0 speed=12.2x    
More than 10000 frames duplicated 227840kB time=00:15:57.07 bitrate=1950.2kbits/s dup=9985 drop=0 speed=  13x    
frame=233748 fps=403 q=-1.0 Lsize= 1826023kB time=02:09:51.58 bitrate=1919.9kbits/s dup=71675 drop=156 speed=13.4x    
video:1695906kB audio:121866kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.453924%
[libx264 @ 0x55ea353e1c80] frame I:943   Avg QP:20.59  size:113752
[libx264 @ 0x55ea353e1c80] frame P:62310 Avg QP:23.35  size: 20851
[libx264 @ 0x55ea353e1c80] frame B:170495 Avg QP:25.47  size:  1936
[libx264 @ 0x55ea353e1c80] consecutive B-frames:  0.7%  5.5%  2.1% 91.7%
[libx264 @ 0x55ea353e1c80] mb I  I16..4: 10.1% 68.7% 21.2%
[libx264 @ 0x55ea353e1c80] mb P  I16..4:  3.4%  8.4%  0.6%  P16..4: 50.6%  9.5% 10.9%  0.0%  0.0%    skip:16.6%
[libx264 @ 0x55ea353e1c80] mb B  I16..4:  0.4%  1.0%  0.0%  B16..8: 20.2%  0.6%  0.1%  direct: 1.6% 

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera


frame=65293 fps=286 q=-1.0 Lsize= 1359807kB time=01:52:05.00 bitrate=1656.4kbits/s dup=17 drop=43791 speed=29.5x    
video:1251224kB audio:105511kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.226422%
[libx264 @ 0x557139948600] frame I:272   Avg QP:17.44  size:154231
[libx264 @ 0x557139948600] frame P:16763 Avg QP:20.32  size: 48868
[libx264 @ 0x557139948600] frame B:48258 Avg QP:23.41  size:  8706
[libx264 @ 0x557139948600] consecutive B-frames:  1.1%  0.7%  0.8% 97.4%
[libx264 @ 0x557139948600] mb I  I16..4: 10.0% 63.5% 26.5%
[libx264 @ 0x557139948600] mb P  I16..4:  3.7% 10.9%  1.1%  P16..4: 41.2% 18.9% 19.1%  0.0%  0.0%    skip: 5.1%
[libx264 @ 0x557139948600] mb B  I16..4:  1.3%  4.0%  0.2%  B16..8: 28.5%  3.3%  0.7%  direct: 9.7%  skip:52.3%  L0:49.0% L1:40.3% BI:10.8%
[libx264 @ 0x557139948600] 8x8 transform intra:70.5% inter:64.3%
[libx264 @ 0x557139948600] coded y,uvDC,uvAC intra: 63.6% 97.8% 77.0% inter: 23.8% 43.8% 12.0%
[libx264 @ 0x557139948600] i16

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/vin_20210527_1010_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/vin_20210527_1010_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera.mov':
  Metadata:
    major_brand     : qt  
    minor_version   : 0
    compatible_brands: qt  
    creation_time   : 2021-05-27T10:22:06.000000Z
    com.apple.quicktime.make: Apple
    com.apple.quicktime.model: MacBookAir7,1
    com.apple.quicktime.software: Mac OS X 10.15.7 (19H2)
    com.apple.quicktime.creationdate: 2021-05-27T10:11:45+0200
  Duration: 02:10:05.12, start: 0.000000, bitrate: 10757 kb/s
  Stream #0:0(und): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(tv, smpte170m/bt709/bt709, progressive), 1280x720 [SAR 1:1 DAR 16:9], 10504 kb/s, 13.75 fps, 29.17 tbr, 30k tbn, 60k tbc (default)
    Metadata:
      creation_time   : 2021-05-27T10:22:06.000000Z
      handler_name    : Core Media Video
      vendor_id       : [0][0][0][0]
      encoder         : H.264
  Stream #0:1(und): Audio: aac (L

In [10]:
# if not os.path.exists('NDSConv_vids'):
#         os.makedirs('NDSConv_vids')

In [9]:
if not os.path.exists('GC_Conv_vids'):
        os.makedirs('GC_Conv_vids')
os.chdir('GC_Conv_vids')

In [10]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids'

In [11]:
#os.listdir(os.path.join(os.path.dirname(to_dir), 'DSConv_vids'))
# os.path.join(os.path.dirname(to_dir), 'NDSConv_vids', os.path.basename(dsv)) 

In [12]:
#cropping video and Gamma correction
def downsize_gc(convid):
    dsv = deeplabcut.DownSampleVideo(convid, width=320, height=240)
    # Load the video file
    video = cv2.VideoCapture(dsv)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #*********************************************

    # Set the start and end points to crop out
    start_sec = 0  # Crop out the first x seconds
    end_sec = 0 # Crop out the last y seconds

    #Set gamma correction value
    gamma = 2

    #**********************************************

    start_frame = int(start_sec * fps)
    end_frame = int((total_frames / fps) - end_sec) * fps

    # Set the video writer
    output_name = os.path.splitext(os.path.basename(convid))[0].lstrip('.') + '_GC.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Loop through the frames and write them to the output video
    for i in range(start_frame, end_frame):
        ret, frame = video.read()
        if not ret:
            break

        # Normalize the pixel values to the range [0, 1]
        frame_normalized = frame.astype(np.float32) / 255.0

        # Apply gamma correction
        frame_corrected = np.power(frame_normalized, 1/gamma)

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

        out.write(frame_scaled)
    new = os.path.join(os.path.dirname(os.path.dirname(dsv)), 'GC_Conv_vids', os.path.basename(dsv))
    shutil.move(dsv, new)

    # Release the video objects
    video.release()
    out.release()
    cv2.destroyAllWindows()

In [15]:
# vid_names = []
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
        downsize_gc(path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [13]:
# os.path.join(os.path.dirname(os.path.dirname(input_file)), 'All_vids/Conv_vids', out_name +'_conv.mp4')
# out_name = os.path.splitext(os.path.basename(path))[0].lstrip('.') + '.mp4'
# out_name

In [14]:
os.chdir('..')
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [15]:
folder = [] 
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids', file))
        folder.append(path)
print(folder)
        
# folder2 = [] 
# for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
#     if file.endswith(".mp4"):
#         path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
#         folder2.append(path)
        
# print(folder, folder2)

['/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210506_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/vin_20210528_1002_XBI19_reversal_learning_foraging_SC3_DCS_frontCameradownsampled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/vin_20210528_1002_XBI19_reversal_learning_foraging_SC3_DCS_frontCamera_GC.mp4', '/home/yramakrishna/DeepLabCut/c

In [4]:
project_name = 'DLC'
your_name = 'VK'

In [6]:
def process(fold, project_name, your_name):
    file = fold[0]
    print(file)
    bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
                 'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
    videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
    video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
    model_options = deeplabcut.create_project.modelzoo.Modeloptions
    model_selection = 'primate_face'

    config_path, train_config_path = deeplabcut.create_pretrained_project(
        project_name,
        your_name,
        video_down,
        videotype=videotype,
        model=model_selection,
        analyzevideo=True,
        createlabeledvideo=False,
        copy_videos=False,
    )

    edits = {
        'dotsize': 1.5,  # size of the dots!
        'pcutoff': 0.4,  # the higher, the more conservative the plotting!
    }
    deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
    project_path = os.path.dirname(config_path)
    full_video_path = []
    for i in fold:
        full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))

    # filter predictions (should already be done above ;):
    deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

    # re-create the video with your edits!
    # deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
    deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
    
    

In [18]:
process(folder, project_name, your_name)

# all_fold = [folder2, folder]
# print(all_fold)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4
Project "/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12" already exists!
Loading.... primate_face


/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12/dlc-models/iteration-0/DLCApr12-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12/dlc-models/iteration-0/DLCApr12-trainset95shuffle1


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-04-12 16:24:54.657313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 16:24:54.660784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 16:24:54.660927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 16:24:54.661505: I tensorflow/core/platfo

Analyzing all the videos in the directory...
Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12/videos/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12/videos/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4
Duration of video [s]:  5032.88 , recorded with  16.01 fps!
Overall # of frames:  80569  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


  0%|                                                                      | 0/80569 [00:00<?, ?it/s]2023-04-12 16:24:57.728861: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2023-04-12 16:24:58.319318: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
100%|█████████████████████████████████████████████████████████| 80569/80569 [02:25<00:00, 553.86it/s]


...

  1%|▎                                                       | 1301/227649 [00:01<03:43, 1011.37it/s]

Duration of video [s]: 16128.23, recorded with 30.0 fps!

  2%|█                                                       | 2479/127609 [00:01<01:31, 1365.52it/s]


Overall # of frames: 483847 with cropped frame dimensions: 320 240
Generating frames and creating video.


100%|██████████████████████████████████████████████████████| 483847/483847 [03:39<00:00, 2202.92it/s]


In [ ]:
# def main():
#     Parallel(n_jobs=1)(delayed(process)(fold, project_name, your_name) for fold in all_fold)


In [ ]:
# main()

In [ ]:

# for fold in all_fold:
#     name_fold = 'Analyse_' + os.path.basename(os.path.dirname(fold[0]))
#     print(name_fold)
#     print(fold)
#     if not os.path.exists(name_fold):
#         os.makedirs(name_fold)
#     os.chdir(name_fold)
#     print(os.getcwd())
#     process(fold, project_name, your_name)
#     os.chdir('..')
#     print(os.getcwd())
    

In [ ]:

# proj_fold

In [5]:
os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids')
h5files = []
vid_to_crop = []
today = str(date.today())
today = "2023-04-12"
proj_fold = project_name+'-'+your_name+'-'+today
# print(today)
target = os.path.join(os.getcwd(),proj_fold,'videos')
print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
# print(h5files, vid_to_crop)
# videotype = os.path.splitext(file)[-1].lstrip('.')

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-12/videos


In [23]:
def crop_it(feed):
    
    fl_pth, vid_pth, x = feed
    # deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))
    print("Starting Cropping")

    # Load tracking results generated by DeepLabCut
    tracking_data = pd.read_hdf(fl_pth)

    # Define the names of the facial features that you want to extract frames for
    feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

    # Define the threshold for the confidence score of the facial features
    confidence_threshold = 0.9
    
    # Load the input video
    cap = cv2.VideoCapture(vid_pth)
    
    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the output video writer
    bn = os.path.basename(vid_pth).split('.')[0]
    out_file = 'AAACrpoutput_video' + bn + '.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_file, fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    print("Output file written - ", out_file)

    # Loop through the video frames and extract frames with facial features
    frame_number = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # Get the tracking data for the current frame
            frame_data = tracking_data.loc[frame_number]

            # Check if the desired facial features are present in the frame
            feature_present = False
            check = 0
            for feature_name in feature_names:
                if feature_name in frame_data.loc[x] and frame_data.loc[x].loc[feature_name].loc['likelihood'] > confidence_threshold:
                    check += 1
            if check==4:
                feature_present = True

            # If the desired facial features are present, save the frame to the output video
            if feature_present: #check==4
                cv2.imshow('output', frame)
                out.write(frame)
            # Display the output
            #cv2.imshow('output', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            print(round(frame_number/tot_frame*100, 2), end = '\r')
            # Increment the frame number
            frame_number += 1
        else:
            break
    print("Done")
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [7]:
os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids')

In [17]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [9]:
if not os.path.exists('Cropped_vids'):
        os.makedirs('Cropped_vids')
os.chdir('Cropped_vids')

In [19]:
crop_source = []
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')
        # print(l)
        # print(i)
        # print(found)
        # print(k)
        # print(l)
        # print('')
        crop_source.append([i,found, l])
c_s = np.array(crop_source)


In [ ]:
# crop_it(c_s[0])
for i in c_s:
    crop_it(i)

Starting Cropping
Output file written -  AAACrpoutput_videovin_20210528_1002_XBI19_reversal_learning_foraging_SC3_DCS_frontCamera_GCDLC_resnet50_DLCApr12shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videovin_20210526_1022_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampledDLC_resnet50_DLCApr12shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videovin_20210527_1010_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera_GCDLC_resnet50_DLCApr12shuffle1_1030000_filtered_labeled.mp4


...

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videoken_20210506_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampledDLC_resnet50_DLCApr12shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videonil_20210614_XBI19downsampledDLC_resnet50_DLCApr12shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread (0xe0f5fa0).
Cannot move to target thread (0xdbee300)

QObject::moveToThread: Current thread (0xdbee300) is not the object's thread

In [67]:
# DLC_resnet50_DLCMar31shuffle1_1030000
# fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGrDLC_resnet50_DLC_GazeXBIMar31shuffle1_1030000_filtered.h5'

# vid_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/DLC_GazeXBI-anc-2023-03-31/videos/CropGrDLC_resnet50_DLC_GazeXBIMar31shuffle1_1030000_filtered_labeled.mp4'

In [ ]:
# shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_video/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5')



In [ ]:


# final_fl_pth = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/CropGrDLC_resnet50_DLC_GazeXBIMar30shuffle1_1030000_filtered.h5
# full_video_path

In [ ]:
# try:
#     shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# try:
#     os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Longdownsampled.mp4')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# import tkinter as tk
# from tkVideoPlayer import TkinterVideo

# root = tk.Tk()

# videoplayer = TkinterVideo(master=root, scaled=True)
# videoplayer.load(final)
# videoplayer.pack(expand=True, fill="both")

# videoplayer.play() # play the video

# root.mainloop()

In [ ]:
# cap = cv2.VideoCapture("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# ret, frame = cap.read()
# while(1):
#    ret, frame = cap.read()
#    cv2.imshow('frame',frame)
#    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
#        cap.release()
#        cv2.destroyAllWindows()
#        break
#    cv2.imshow('frame',frame)

In [ ]:
# from moviepy.editor import *
# import pygame
 
# clip = VideoFileClip('/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4')
# clip.preview()
 
# pygame.quit()

In [ ]:
# from tkinter import *
# from tkvideo import tkvideo

# root = Tk()
# my_label = Label(root)
# my_label.pack()
# player = tkvideo("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4", my_label, loop = 1, size = (1280,720))
# player.play()

# root.mainloop()

In [ ]:
# dlc_model = config_path
# dlc_config = deeplabcut.load_config(config_file)
# dlc_model = deeplabcut.getModel(dlc_config)


# # Load the video file
# cap = cv2.VideoCapture(video_down)

# fps = int(cap.get(cv2.CAP_PROP_FPS))

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('CropMFace.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# # Loop over each frame in the video
# frame_count = 0
# video.set(cv2.CAP_PROP_POS_FRAMES, 0)

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     # Break the loop when the video ends
#     if not ret:
#         break

#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']

#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         out.write(frame_scaled)
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))

#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1


#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# out.release()
# cap.release()
# cv2.destroyAllWindows()


In [ ]:

# import cv2
# import numpy as np
# import deeplabcut

# # Load the DeepLabCut model
# config_file = 'path/to/config.yaml'
# dlc_model = deeplabcut.load_model(config_file)

# # Load the video file
# cap = cv2.VideoCapture('path/to/video')

# # Loop over each frame in the video
# frame_count = 0
# while(cap.isOpened()):
#     ret, frame = cap.read()
    
#     # Break the loop when the video ends
#     if not ret:
#         break
    
#     # Detect the monkey face using the DeepLabCut model
#     dlc_coordinates, _ = dlc_model.predict(frame, tracking=True)
#     monkey_face_coordinates = dlc_coordinates['monkey_face']
    
#     # Check if monkey face is detected in the frame
#     if monkey_face_coordinates.any():
#         # Get the bounding box coordinates of the monkey face
#         x, y, w, h = cv2.boundingRect(monkey_face_coordinates.astype(int))
        
#         # Crop out the monkey face and save it to a new file
#         monkey_face = frame[y:y+h, x:x+w]
#         cv2.imwrite(f'monkey_face_{frame_count}.jpg', monkey_face)
#         frame_count += 1
    
#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the video capture and close all windows
# cap.release()
# cv2.destroyAllWindows()
